In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import requests
import sys

In [2]:
imbd_data = pd.read_csv('final_voting_yeah.csv',index_col=0)

In [3]:
text_data = imbd_data.loc[: , ['Director', 'Writer', 'First_actor', 'Second_actor', 'Third_actor', 'Forth_actor']]

In [4]:
from collections import Counter

In [5]:
DIRECTOR_W = 4
WRITER_W = 1
ACTOR_W = 4
ACTOR_W_1 = 2

In [6]:
weight_dict = {
    'Director':DIRECTOR_W,
    'Writer':WRITER_W,
    'First_actor':ACTOR_W,
    'Second_actor':ACTOR_W,
    'Third_actor':ACTOR_W_1,
    'Forth_actor':ACTOR_W_1
}

In [7]:
def get_weight(type):
    if type == 'Director':
        return word_count(text_data[type].tolist(), DIRECTOR_W)
    elif type == 'Writer':
        return word_count(text_data[type].tolist(), WRITER_W)
    elif type == 'First_actor' or type == 'Second_actor':
        return word_count(text_data[type].tolist(), ACTOR_W)
    else: return word_count(text_data[type].tolist(), ACTOR_W_1)
    
def word_count(name_list, adj_fct):
    count = Counter()
    for name in name_list:
        if type(name) is str: count[name.strip()] += 1 * adj_fct
    return count

def conbime_act():
    c1 = get_weight('First_actor')
    c2 = get_weight('Second_actor')
    c3 = get_weight('Third_actor')
    c4 = get_weight('Forth_actor')
    keys = [*c1.keys()] + [*c2.keys()]  + [*c3.keys()] + [*c4.keys()]   
    m_dic = dict.fromkeys(keys)
    for key in m_dic.keys():
        count = 0
        if key in c1.keys(): count += c1[key]
        if key in c2.keys(): count += c2[key]
        if key in c3.keys(): count += c3[key]
        if key in c4.keys(): count += c4[key]
        m_dic[key] = count
    return m_dic

In [8]:
alpha_smoothing = 0.005 

In [ ]:
get_weight(role)

In [9]:
get_weight('First_actor')

Counter({'Sam Worthington': 20,
         'Leonardo DiCaprio': 72,
         'Gal Gadot': 4,
         'Tobey Maguire': 36,
         'Sam Neill': 28,
         'Ewan McGregor': 56,
         'Tom Hanks': 140,
         'Alan Howard': 4,
         'Macaulay Culkin': 16,
         'Ray Anthony': 4,
         'Jim Carrey': 72,
         'Tommy Lee Jones': 60,
         'Helen Hunt': 24,
         'Tom Cruise': 108,
         'Matt Damon': 80,
         'Jackie Chan': 56,
         'Robby Benson': 8,
         'Jason Lee': 28,
         'Mike Myers': 44,
         'Bruce Willis': 132,
         'Brandon Routh': 12,
         'Kristen Stewart': 28,
         'Brad Pitt': 72,
         'Paul Rudd': 44,
         'Hugh Jackman': 52,
         'Will Smith': 60,
         'Richard Gere': 92,
         'Ray Romano': 8,
         'Nicolas Cage': 152,
         'Russell Crowe': 64,
         'Anthony Hopkins': 84,
         'Michael Keaton': 72,
         'Dave Foley': 8,
         'Adam Sandler': 112,
         'Brendan Fraser':

In [13]:
1/np.log(len(imbd_data)/get_weight('First_actor')['Leonardo DiCaprio'])

0.1984333814765706

In [12]:
1/np.log(len(imbd_data)/get_weight('First_actor')['Ryan Newman'])

0.12610584692400806

In [14]:
def normed_adj_weight(name, role):
    soc_loc = ['imdbRating', 'Internet_Movie_Database', 'Rotten_Tomatoes', 'Metacritic']
    col = role
    if role == 'First_actor' or role == 'Second_actor' or role == 'Third_actor' or role == 'Forth_actor':
        col = ['First_actor' , 'Second_actor', 'Third_actor', 'Forth_actor']
        score = 0
        leng = 0
        for co in col:
            idc = text_data[co][text_data[col][text_data[col] == name].notna()[co]].index.values.tolist()
            for id in idc:
                sco_of_film = (imbd_data.iloc[id][['imdbRating', 'Internet_Movie_Database', 'Rotten_Tomatoes', 'Metacritic']]).mean()
                sum_of_weight = sum([weight_dict[item] for item in weight_dict if type(imbd_data.iloc[id][item]) is str])
                base = np.log(sco_of_film)
                score += base + sco_of_film*(weight_dict[co])/sum_of_weight
            leng += len(idc)
        smoothing_weight = (1 + alpha_smoothing)/(leng + len(imbd_data)*alpha_smoothing)
        final_socre = score*smoothing_weight*(5/np.log(len(imbd_data)/conbime_act()[name]))
    else: 
        idx = text_data[col][text_data[col] == name].index.values.tolist()
        score = 0
        for id in idx:
            sco_of_film = (imbd_data.iloc[id][['imdbRating', 'Internet_Movie_Database', 'Rotten_Tomatoes', 'Metacritic']]).mean()
            sum_of_weight = sum([weight_dict[item] for item in weight_dict if type(imbd_data.iloc[id][item]) is str])
            base = np.log(sco_of_film)
            score += base + sco_of_film*(weight_dict[role])/sum_of_weight
        smoothing_weight = (1 + alpha_smoothing)/(len(idx) + len(imbd_data)*alpha_smoothing)
        final_socre = score*smoothing_weight*(5/np.log(len(imbd_data)/get_weight(role)[name]))
    return final_socre

In [ ]:
score_list = []
for i in range(len(imbd_data)):
    score_per_film = 0
    for col in ['Director', 'Writer', 'First_actor', 'Second_actor', 'Third_actor', 'Forth_actor']:
        if type(imbd_data.iloc[i][col]) is str:
            score_per_film+= normed_adj_weight(imbd_data.iloc[i][col], col)
    score_list.append(score_per_film)

In [22]:
#score_Director = []
for i in range(5079, len(imbd_data)):
    score_per_film = 0
    for col in ['First_actor', 'Second_actor', 'Third_actor', 'Forth_actor']:
        if type(imbd_data.iloc[i][col]) is str:
            score_per_film+= normed_adj_weight(imbd_data.iloc[i][col], col)
    score_Director.append(score_per_film)

In [23]:
len(score_Director)

11116

In [24]:
len(my_data)

NameError: name 'my_data' is not defined

In [25]:
imbd_data['score_of_A'] = score_Director

In [26]:
my_data = imbd_data.drop(['Director', 'Writer', 'First_actor', 'Second_actor', 'Third_actor', 'Forth_actor', 'imdbRating', 'Internet_Movie_Database', 'Rotten_Tomatoes', 'Metacritic'], axis = 1)

In [27]:
my_data.to_csv('final_score_rev_2.csv')

In [28]:
my_data

,BoxOffice,Runtime,Title,Year,imdbVotes,Action,Adventure,Animation,Biography,Comedy,...,Short,Sport,Thriller,War,Western,Country,Country_count,score_of_D,score_of_W,score_of_A
0,8.838271e+08,162,Avatar,2009,1010278,1,1,0,0,0,...,0,0,0,0,0,"UK, USA",2,0.315535,0.190258,2.561038
1,1.039048e+09,194,Titanic,1997,922729,0,0,0,0,0,...,0,0,0,0,0,USA,1,0.315535,0.190258,3.056170
2,4.254846e+08,141,Wonder Woman,2017,444177,1,1,0,0,0,...,0,0,0,1,0,"USA, China, Hong Kong",3,0.093359,0.024266,1.783530
3,5.675917e+08,121,Spider-Man,2002,622899,1,1,0,0,0,...,0,0,0,0,0,USA,1,0.436842,0.074590,3.533636
4,7.926653e+07,127,Jurassic Park,1993,755018,0,1,0,0,0,...,0,0,1,0,0,USA,1,1.454859,0.236569,2.472766
5,4.923462e+08,140,Star Wars: Episode III - Revenge of the Sith,2005,619959,1,1,0,0,0,...,0,0,0,0,0,"USA, Italy, Switzerland, Thailand, Tunisia",5,0.238646,0.174334,2.600728
6,5.632720e+08,142,Forrest Gump,1994,1539440,0,0,0,0,0,...,0,0,0,0,0,USA,1,0.797230,0.024786,2.627257
7,4.485203e+08,178,The Lord of the Rings: The Fellowship of the Ring,2001,1451077,0,1,0,0,0,...,0,0,0,0,0,"New Zealand, USA",2,0.563321,0.085386,1.114579
8,3.393202e+08,148,Inception,2010,1767563,1,1,0,0,0,...,0,0,1,0,0,"USA, UK",2,0.547427,0.109988,3.086924
9,5.529723e+08,103,Home Alone,1990,386574,0,0,0,0,1,...,0,0,0,0,0,USA,1,0.523540,0.479934,2.088868
